# Coherence Traffic Analysis
## Migratory Sharing

In [ ]:
import itertools

### Functions to generate the all sequences and calculate the average message count

In [47]:

def add_loads(permutations_all):
    p_all=[]
    j=0
    for a in permutations_all:
        p_all.append([])
        for i in range(len(a)*2):
            if i%2 !=0:
                p_all[j].append(a[int((i-1)/2)])
            else:
                p_all[j].append("l")
        j+=1 
    return p_all

def generate(N,scribble_percent):
    procs = N
    elements=[]
    for i in range(int(scribble_percent*N)):
        elements.append('S')
    for i in range(int((1-scribble_percent)*N)):
        elements.append('St')
    # print(elements) 
    permutations = itertools.permutations(elements, procs)
    permutations_all =  set(permutations) 
    return permutations_all

def find_msg_count(p_all):
    messages = []
    for p in p_all:
        sharers = 0
        msg_count = 0
        isModified = False
        # print(p)
        for i in p:
            tmp_count = 0
            if i == "l":
                if sharers >= 0 and not isModified:
                    sharers += 1
                    tmp_count += 2
                if isModified:
                    sharers += 2
                    tmp_count += 4
                    isModified = False
            if i == "St":
                tmp_count += (sharers-1)*3
                tmp_count += 2
                sharers = 0
                isModified = True
            if i == "S":
                tmp_count += 0
            msg_count += tmp_count
            # print(i,tmp_count,msg_count)
        messages.append(msg_count)
    return messages

def find_idx(series,p_all,messages):
    tmp = []
    for i in range(len(series)*2):
        if i%2 !=0:
            tmp.append(series[int((i-1)/2)])
        else:
            tmp.append("l")
    idx = p_all.index(tmp)
    return messages[idx]

def find_mean_msg_count(proc,percent):
    k= generate(proc,percent)
    p_all = add_loads(k)
    messages = find_msg_count(p_all)
    return sum(messages)/len(messages)



Test performed to check correctness

In [48]:

k= generate(8,0.5)
p_all = add_loads(k)
messages = find_msg_count(p_all)


test = ['St', 'S', 'St', 'St', 'S', 'S', 'S', 'St' ]

print(find_idx(test,p_all,messages))

print(find_mean_msg_count(8,0.5))


51
49.6


### calculation of the traffic for various procs and scribble percentages

In [49]:
procs = [4,8]
scribble_percentages = [0,0.25,0.5,0.75,1]
mean_msg_count_ms = []
j=0
for proc in procs:
    mean_msg_count_ms.append([])
    for i in scribble_percentages:
        mean_msg_count_ms[j].append(find_mean_msg_count(proc,i))
        # x=generate(proc,i)
        # print(x)
    j+=1
mean_msg_count_ms


[[31.0, 26.75, 22.0, 16.0, 8.0], [67.0, 58.642857142857146, 49.6, 38.5, 16.0]]

## Producer Consmer Sharing

### Functions to generate and compute the average message count

In [53]:
def generate_pc(N, scribble_percentage):
    l = []
    permutation = itertools.combinations(range(N),int(N*scribble_percentage))
    iter=0
    for k in list(permutation):
        l.append([])
        for i in range(N):
            for j in range(N):
                if(i==j):
                    if(i in k):
                        l[iter].append('S')
                    else:
                        l[iter].append('St')
                else:
                    l[iter].append('l')
        iter+=1
    
    return l

def find_msg_count_pc(p_all,N):
    messages = []
    for s in p_all:
        m=0
        sharers = 0
        msg_count = 0
        isModified = False
        sharing_status = [0]*N
        for k in range(0,len(s),N):
            per_cycle = s[k:k+N]
            for i in range(N):
                tmp_count = 0
                if m >= N:
                    m =0
                k = i+m
                if k >=N:
                    k = k%N
                
                if per_cycle[k] == "l":
                        if sharers >= 0 and not isModified:
                            if(sharing_status[k] == 0):
                                sharers += 1
                                tmp_count += 2
                                sharing_status[k] = 1
                        if isModified:
                            sharers += 2
                            tmp_count += 4
                            isModified = False
                            sharing_status[k] = 1
                            sharing_status[k-1] = 1
                if per_cycle[k] == "St":
                    if sharers >0:
                        tmp_count += (sharers-1)*3
                    tmp_count += 2
                    sharing_status = [0]*N
                    sharers = 0
                    isModified = True
                if per_cycle[k] == "S":
                    tmp_count += 0
                msg_count += tmp_count
            m +=1
        messages.append(msg_count)
    return messages

    
    
def find_mean_msg_count_pc(proc,percent):
    k= generate_pc(proc,percent)
    messages = find_msg_count_pc(k,proc)
    # print(messages)
    return sum(messages)/len(messages)



In [54]:
procs = [4,8,16]
scribble_percentages = [0,0.25,0.5,0.75,1]
mean_msg_count = []
j=0
for proc in procs:
    mean_msg_count.append([])
    for i in scribble_percentages:
        mean_msg_count[j].append(find_mean_msg_count_pc(proc,i))
    mean_msg_count[j]
    j+=1
mean_msg_count

[[67.0, 51.0, 35.833333333333336, 21.5, 8.0],
 [291.0, 221.17857142857142, 152.07142857142858, 83.67857142857143, 16.0],
 [1219.0, 921.25, 624.1666666666666, 327.75, 32.0]]